In [1]:
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite
import pandas as pd

In [3]:
model = read_sbml_model('data/iML1515.xml')

In [4]:
model

Name,iML1515
Memory address,0x07f5dad3c5a30
Number of metabolites,1877
Number of reactions,2712
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


1st strategy: Change specificity of phenylalanine hydroxylase and use it to hydrolase tryptophan

In [5]:
#look for metabolites in the model and genes to knock out


for gene in model.genes.query('tna', 'name'):
    print(gene.name, gene.id)

tnaA b3708
tnaB b3709


In [6]:
#gene responsible for 5-HTP and trp degradation

model.genes.b3708

Gene identifier,b3708
Name,tnaA
Memory address,0x07f5dfdecac10
Functional,True
In 3 reaction(s),"SERD_L, TRPAS2, CYSDS"


In [7]:
#trp and 5-HTP degradation reaction

model.reactions.TRPAS2

Reaction identifier,TRPAS2
Name,Tryptophanase (L-tryptophan)
Memory address,0x07f5dfdb36d60
Stoichiometry,h2o_c + trp__L_c <=> indole_c + nh4_c + pyr_c H2O H2O + L-Tryptophan <=> Indole + Ammonium + Pyruvate
GPR,b3708
Lower bound,-1000.0
Upper bound,1000.0


In [8]:
'''
Metabolites present in the model:

Tetrahydromonapterin thmnp_c
L-Tryptophan trp__L_c
Dihydromonapterin dhmpt_c
Indole indole_c
Ammonium nh4_c
Pyruvate pyr_p

Gene encoding 5-HTP degradation

tnaA b3708
'''

'\nMetabolites present in the model:\n\nTetrahydromonapterin thmnp_c\nL-Tryptophan trp__L_c\nDihydromonapterin dhmpt_c\nIndole indole_c\nAmmonium nh4_c\nPyruvate pyr_p\n\nGene encoding 5-HTP degradation\n\ntnaA b3708\n'

1. 5-HTP production (phhA):
tetrahydromonapterin (MH4) + oxygen + tryptophan (trp) <-> 4a-hydroxytetrahydromonopterin (hydro-MH4) + 5-hydroxytryptophan (5-HTP)

In [9]:
#add metabolites to the system
four_alpha_hydroxytetrahydromonopterin = Metabolite(id='4a_hthpth_c', compartment='c')
hydroxytryptophan = Metabolite(id='htrp_c', compartment='c')

In [10]:
#design reaction
phhA = Reaction('phhA')
phhA.add_metabolites({model.metabolites.thmnp_c: -1,
                      model.metabolites.o2_c: -1,
                      model.metabolites.trp__L_c: -1,
                      four_alpha_hydroxytetrahydromonopterin: 1,
                      hydroxytryptophan: 1
                     })
print(phhA.build_reaction_string())

o2_c + thmnp_c + trp__L_c --> 4a_hthpth_c + htrp_c


In [11]:
#add reaction to the system
model.add_reactions([phhA])
model.reactions.phhA

Reaction identifier,phhA
Name,
Memory address,0x07f5dfcfe0400
Stoichiometry,o2_c + thmnp_c + trp__L_c --> 4a_hthpth_c + htrp_c O2 O2 + Tetrahydromonapterin + L-Tryptophan --> +
GPR,
Lower bound,0.0
Upper bound,1000.0


In [12]:
five_hydroxytryptophan_exchange = Reaction('EX_htrp')
five_hydroxytryptophan_exchange.add_metabolites({model.metabolites.htrp_c: -1})
model.add_reaction(five_hydroxytryptophan_exchange)

2. Restioration of MH4

pterin-4alpha-carbinolamine dehydratase (PCD): 4a-hydroxytetrahydromonopterin (hydro-MH4) <-> dihydromonapterin (MH2) + water

In [13]:
#design reaction
pcd = Reaction('pcd')
pcd.add_metabolites({four_alpha_hydroxytetrahydromonopterin: -1,
                      model.metabolites.dhmpt_c: 1,
                      model.metabolites.h2o_c: 1,
                     })
print(pcd.build_reaction_string())

4a_hthpth_c --> dhmpt_c + h2o_c


In [14]:
#add reaction to the system
model.add_reactions([pcd])
model.reactions.pcd

Reaction identifier,pcd
Name,
Memory address,0x07f5dfcf86370
Stoichiometry,4a_hthpth_c --> dhmpt_c + h2o_c --> Dihydromonapterin + H2O H2O
GPR,
Lower bound,0.0
Upper bound,1000.0


dihydromonapterin reductase (DHMR): dihydromonapterin (MH2) + H+ + NADPH <-> NADP+ + tetrahydromonapterin (MH4)

In [15]:
#design reaction
dhmr = Reaction('dhmr')
dhmr.add_metabolites({model.metabolites.h_c: -1,
                      model.metabolites.dhmpt_c: -1,
                      model.metabolites.nadph_c: -1,
                      model.metabolites.nadp_c: 1,
                      model.metabolites.thmnp_c: 1
                     })
print(dhmr.build_reaction_string())

dhmpt_c + h_c + nadph_c --> nadp_c + thmnp_c


In [16]:
# check if the growth is possible with the reactions added
model.optimize()

,fluxes,reduced_costs
CYTDK2,0.00000,-7.523353e-03
XPPT,0.00000,-1.504671e-02
HXPRT,0.00000,-1.504671e-02
NDPK5,0.00000,0.000000e+00
SHK3Dr,0.33424,-2.775558e-17
...,...,...
FESD2s,0.00000,4.996004e-16
OCTNLL,0.00000,8.673617e-19
phhA,0.00000,-6.582934e-02
EX_htrp,0.00000,0.000000e+00


In [18]:
# change objective value to production of 5-HTP, check if E.coli still grows
with model:
    model.objective = model.reactions.phhA
    max_5HTP_production = model.optimize().objective_value
    print(max_5HTP_production)

4.279204339963833


In [19]:
from cobra.flux_analysis import flux_variability_analysis

In [20]:
flux_variability_analysis(model)

,minimum,maximum
CYTDK2,0.000000,-7.083372e-13
XPPT,0.000000,-3.541686e-13
HXPRT,0.000000,-3.541686e-13
NDPK5,-0.022948,-2.833349e-12
SHK3Dr,0.334240,3.342403e-01
...,...,...
FESD2s,0.000000,0.000000e+00
OCTNLL,0.000000,1.506554e-14
phhA,0.000000,4.446426e-14
EX_htrp,0.000000,4.446426e-14


In [21]:
hydroroxy_trp_flux_analysis = flux_variability_analysis(model, model.reactions.EX_htrp, fraction_of_optimum=0.1)
print(hydroroxy_trp_flux_analysis)

         minimum   maximum
EX_htrp      0.0  3.852313


OPTIONAL: INTRODUCTION OF TNA GENE DELETION

Tryptophanase enzyme is knocked out, to inhibit 5-HTP degradation

In [22]:
#gene responsible for 5-HTP and trp degradation

model.genes.b3708

Gene identifier,b3708
Name,tnaA
Memory address,0x07f5dfdecac10
Functional,True
In 3 reaction(s),"SERD_L, TRPAS2, CYSDS"


In [23]:
#trp and 5-HTP degradation reaction

model.reactions.TRPAS2

Reaction identifier,TRPAS2
Name,Tryptophanase (L-tryptophan)
Memory address,0x07f5dfdb36d60
Stoichiometry,h2o_c + trp__L_c <=> indole_c + nh4_c + pyr_c H2O H2O + L-Tryptophan <=> Indole + Ammonium + Pyruvate
GPR,b3708
Lower bound,-1000.0
Upper bound,1000.0


In [24]:
#checking if gene is essential

with model as tnaA_mutant:
    tnaA_mutant.genes.b3708.knock_out()
    print(tnaA_mutant.optimize())

<Solution 0.877 at 0x7f5dfd035be0>


In [26]:
#optimize model with knocked out gene
with model:
    model.genes.b3708.knock_out()
    model.objective = model.reactions.phhA
    max_5HTP_production = model.optimize().objective_value
    print(max_5HTP_production)

4.130413625304133
